In [1]:
# pip install watermark
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,sklearn -g 
#,statsmodels

CPython 3.6.7
IPython 7.4.0

numpy 1.16.2
scipy 1.2.1
pandas 0.24.2
matplotlib 3.0.3
sklearn 0.20.3

compiler   : GCC 8.2.0
system     : Linux
release    : 4.9.93-boot2docker
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


In [2]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
#from glob import glob
import os
#import pickle
#pip install tqdm
#from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
#from scipy.sparse import csr_matrix
#import json
import math
#import seaborn as sns
#import matplotlib
#import matplotlib.pyplot as plt
#import matplotlib.ticker
#%matplotlib inline
import datetime
#from datetime import datetime, date, time,timedelta
import re

In [83]:
COMPARATOR_AUTHOR = "Anton Tushev" 
COMPARATOR_VERSION = "v.06"
PATH_TO_DATA = '/notebooks/pinda/data'
PATH_TO_STRUCG = '/notebooks/pinda/data/alcanc26_03_2019'
FILE_NAME_DATA1 = 'TX04_fixed_v1.csv'
#FILE_NAME_DATA1 = 'TX04_test.csv'
#FILE_NAME_DATA1 = 'test.csv'
IGNORE_CHART = ['@SIMD']
SKIP_EQUAL = True
SKIP_BLOCK_NEXST = True
SKIP_SIGNAL_NFND = True
SKIP_TYPE_STR = True
SKIP_CHART_NFND = True
COMPARATOR_README = "===================================================================================\r\n\
Comparator TDC <=> SimadynD software version: %s \r\nAuthor: %s \r\nComparation result made from %s \
file \r\nDate and time: %s \r\nUse filters: \r\nXXXXXXXX - critical points \
\r\n!!!!!!!! - not critical errors \r\nFalse - difference in programs\r\nTrue - no difference in programs \
\r\n=====================\r\nCurren filters:\r\nSKIP_EQUAL:\t\t%s\r\nSKIP_BLOCK_NEXST:\t%s\r\nSKIP_SIGNAL_NFND:\t%s\r\n\
SKIP_TYPE_STR:\t\t%s\r\nSKIP_CHART_NFND:\t%s\r\n\
===================================================================================\r\n\
"%(COMPARATOR_VERSION,COMPARATOR_AUTHOR,FILE_NAME_DATA1,datetime.datetime.now(),SKIP_EQUAL,SKIP_BLOCK_NEXST, \
   SKIP_SIGNAL_NFND,SKIP_TYPE_STR,SKIP_CHART_NFND)
print(COMPARATOR_README)


Comparator TDC <=> SimadynD software version: v.06 
Author: Anton Tushev 
Comparation result made from TX04_fixed_v1.csv file 
Date and time: 2019-05-21 03:37:31.913177 
Use filters: 
XXXXXXXX - critical points 
!!!!!!!! - not critical errors 
False - difference in programs
True - no difference in programs 
Curren filters:
SKIP_EQUAL:		True
SKIP_BLOCK_NEXST:	True
SKIP_SIGNAL_NFND:	True
SKIP_TYPE_STR:		True
SKIP_CHART_NFND:	True



In [84]:
data_a1 = pd.read_csv(os.path.join(PATH_TO_DATA,FILE_NAME_DATA1),keep_default_na=False,na_values=[''], encoding="cp1251",sep=';',header=None)
#assign columns
data_a1 = data_a1[[0,1,3,5,11,28,29]]
data_a1.columns = ['rack_cpu','chart','block','signal','value','type','out']
data_a1 = pd.concat([data_a1, data_a1['rack_cpu'].str.split("\\",expand = True)[[0,1]]], axis=1) 
data_a1.columns = ['rack_cpu','chart','block','signal','value','type','out','rack','cpu']
data_a1["out"]= data_a1["out"].replace(("IN","OUT"),(0,1)).astype('bool') 
data_a1 = data_a1[~data_a1['chart'].isin(IGNORE_CHART)]
data_a1["block"] = data_a1["block"].astype(str)
print(data_a1['type'].unique())
print(data_a1['chart'].unique())
#data_a1[(data_a1['chart']=="RACK_1")&(data_a1['block']=="VM1")]
data_a1

['BOOL' 'REAL' 'SDTIME' 'WORD' 'INT' 'DINT' 'GLOBAL' 'STRING' 'DWORD'
 'BYTE']
['ENCC_1' 'ENBRU1' 'ENBRI1' 'ENBEA1' 'BGT__1' '@SEND1' 'ENGAU1' 'ENGN11'
 'ENMB_1' 'ENOSW1' 'ENGUA1' 'ENHDR1' 'ENIHW1' 'ENINS1' 'ENEXT1' 'ENENT1'
 'ENSR_1' 'ENSPH1' 'ENSCD1' 'ENPIN1' 'ENPEL1' 'ENPAR1' 'ENISW1' 'ENLAP1'
 'ENMA_1' 'ENMAN1' 'ENOHW1' 'ENSTR1' 'ENTEM1' 'ENTUR1' 'MESY_1' 'MONIM1'
 'MONIW1' 'SERV_1' 'VISU_1' 'WPS__1' 'ENISW_MSU' 'ENDIA1' 'ENCRA1'
 'ENCPT1' 'ENCPS1' 'ENCPP1' 'ENCPM1' 'ENCPH1' 'ENCPC1' 'ENCPA1' 'ENCOO1'
 'ENMG_1' 'ENMC_1' 'ENMD_1' 'ENME_1' 'ENMF_1' '@SEND2' 'EXBAN2' 'EXBBH2'
 'EXBBY2' 'EXBEA2' 'EXBW_2' 'EXBWA2' 'EXBWT2' 'EXCC_2' 'EXCRA2' 'EXCRM2'
 'EXCUT2' 'EXDEF2' 'EXDIA2' 'EXFEL2' 'EXGAU2' 'EXGN12' 'EXGN22' 'EXIHW2'
 'EXIR_2' 'EXIRP2' 'EXISW2' 'EXKNA2' 'EXLAP2' 'EXMAN2' 'EXMLR2' 'EXMLT2'
 'EXOHW2' 'EXOSW2' 'EXPAR2' 'EXPUS2' 'EXREJ2' 'EXREL2' 'EXSC_2' 'EXSCA2'
 'EXSCM2' 'EXSH12' 'EXSH22' 'EXSHJ2' 'EXSHT2' 'EXSML2' 'EXSR_2' 'EXSRC2'
 'EXSRE2' 'EXSSC2' 'EXSSR2' 'EXSTR2' 'EXTB_2' 'EXTB

,rack_cpu,chart,block,signal,value,type,out,rack,cpu
0,TX04\D01P01\,ENCC_1,ABLOKD,I1,NaN,BOOL,False,TX04,D01P01
1,TX04\D01P01\,ENCC_1,ABLOKD,I2,NaN,BOOL,False,TX04,D01P01
2,TX04\D01P01\,ENCC_1,ABLOKD,Q,0,BOOL,True,TX04,D01P01
3,TX04\D01P01\,ENCC_1,ADD100,X1,NaN,REAL,False,TX04,D01P01
4,TX04\D01P01\,ENCC_1,ADD100,X2,NaN,REAL,False,TX04,D01P01
5,TX04\D01P01\,ENCC_1,ADD100,Y,0,REAL,True,TX04,D01P01
6,TX04\D01P01\,ENCC_1,ADPOF2,I1,NaN,BOOL,False,TX04,D01P01
7,TX04\D01P01\,ENCC_1,ADPOF2,I2,NaN,BOOL,False,TX04,D01P01
8,TX04\D01P01\,ENCC_1,ADPOF2,I3,NaN,BOOL,False,TX04,D01P01
9,TX04\D01P01\,ENCC_1,ADPOF2,I4,NaN,BOOL,False,TX04,D01P01


In [86]:
def get_some_value(rack,chart,block,signal,signal_type,PATH_TO_STRUCG=PATH_TO_STRUCG): 
    try:
        with open(os.path.join(PATH_TO_STRUCG,rack.lower(),chart.lower()+'.cfp')) as f:
            lines = [line.rstrip('\n') for line in f]
    except EnvironmentError: 
        try:
            with open(os.path.join(PATH_TO_STRUCG,rack.lower(),chart.lower()+'.ofp')) as f:
                lines = [line.rstrip('\n') for line in f]
        except EnvironmentError: 
            return -4,-1
    
    b = re.compile(r"\d+ ?%s +:"%(block))
    sf = re.compile(r"^ *\d+\s+%s\s+"%(signal))
    s = re.compile(r'(([<-] ?)|(@TYP=..),)([^@"\',&]+)([,"\'&]|$)')
    s_string = re.compile(r'(([-])|(@TYP=..),) +(\'.+\')([ ,"\&]|$)')
    el = re.compile(r'^ *\d+\s+[+]')
    test_signal_no_zero = re.compile(r"[a-zA-Z_][0-9]$")
    if(test_signal_no_zero.search(signal)):
        signal_no_zero_check = True
        signal2 = signal[:-1]+"0"+signal[-1:]
        sf2 = re.compile(r"^ *\d+\s+%s\s+"%(signal2))
    else:
        signal_no_zero_check = False
    mark_search_signal = False
    mark_block_found = False
    mark_signal_found = False
    for line in lines:
        if block in line:
            if b.search(line):
                mark_search_signal = True
                mark_block_found = True
                #print("Block found >>> ", line)
                continue
        else:
            if mark_search_signal:
                if signal_no_zero_check:
                    cnd = bool(sf.search(line) or bool(sf2.search(line)))
                else:
                    cnd = bool(sf.search(line))
                if cnd:
                    mark_search_signal = False
                    if signal_type == "STRING":
                        m=s_string.search(line)
                    else:
                        m=s.search(line)
                    if m:
                        #return normal result
                        #print(m.group(4),m.group(3))
                        return m.group(4),m.group(3)
                    else:
                        #print('ERROR Signal can not be recognized!')
                        return -3,-1
                elif el.search(line):
                    return -2,-1  
    if not mark_block_found:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ERROR No block found!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return -1,-1
    if mark_search_signal:
        #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!ERROR No signal found!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        return -2,-1
    return -1,-1
#@TYP=V2
#1219	X1  N2 < @TYP=O2,5,'Manual'
#value = get_some_value("TX03","EMUL_1","YS__03","X0")
#print("Comparison:      ",j['value'], ">>>",value)

In [20]:
##############TEST CELL###########NOT USED#######################################
tst_line =   "1077	X   NF < @TYP=TF,300.0[S]"
s = re.compile(r'(([<-] ?)|(@TYP=..),)([^@"\',&]+)([,"\'&]|$)')
#con_check = re.compile(r'(\S+[.]\D+)|(^\$\S+)')
m=s.search(tst_line)
print(m)
print(m.group(4))
print(m.group(3))


<_sre.SRE_Match object; span=(14, 30), match='@TYP=TF,300.0[S]'>
300.0[S]
@TYP=TF


In [69]:
##############TEST CELL###########NOT USED#######################################
tst_line = " 285	I2  B1 < 0"
s = re.compile(r"^ *\d+\s+I2\s+")
m=s.search(tst_line)
print(m)

<_sre.SRE_Match object; span=(0, 9), match=' 285\tI2  '>


In [87]:
def round_half_down(n, decimals=5):
    multiplier = 10 ** decimals
    return math.ceil(n*multiplier - 0.5) / multiplier

def twos_complement(hexstr,bits,base=16):
    value = int(hexstr,base)
    if value & (1 << (bits-1)):
        value -= 1 << bits
    return value

def toDeci(str,base): 
    def val(c): 
        if c >= '0' and c <= '9': 
            return ord(c) - ord('0') 
        else: 
            return ord(c) - ord('A') + 10; 
    llen = len(str) 
    power = 1 #Initialize power of base 
    num = 0     #Initialize result 
  
    # Decimal equivalent is str[len-1]*1 +  
    # str[len-1]*base + str[len-1]*(base^2) + ...  
    for i in range(llen - 1, -1, -1): 
          
        # A digit in input number must  
        # be less than number's base  
        if val(str[i]) >= base: 
            print('Invalid Number') 
            return -1
        num += val(str[i]) * power 
        power = power * base 
    return num 

def compare_values(val_tdc,val_sim,val_type,subtype):
    #['BOOL' 'INT' 'WORD' 'SDTIME' 'GLOBAL' 'STRING' 'REAL' 'DINT' 'BYTE' 'DWORD']
    #check form of function output
    con_check = re.compile(r'(\S+[.]\D+)|(^\$\S+)')
    if con_check.search(val_sim):
        #print("Signal value of simadyn has chart connection")
        return -1
    if val_type=='REAL':
        #print(val_tdc)
        v_t = round_half_down(float(val_tdc.replace(',','.')),decimals=6)
        
        units = 1   
        if "[" in val_sim:
            un_txt = val_sim.lower()
            un_txt = un_txt.split("[")[1]
            un_txt = un_txt.split("]")[0]
            #seems s/min/ms conversion works even with REAL types...
            if subtype=="@TYP=TF":
                if un_txt=="s":
                    units = 1000
                elif un_txt=="min":
                    units = 60000
                elif un_txt=="ms":
                    units = 1  
            val_sim = val_sim.split("[")[0]
        if '%' in val_sim:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = round_half_down(float(tmp_txt)*units*0.01,decimals=6)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = round_half_down(float(tmp_txt)*units,decimals=6)
    elif val_type=='BOOL':
        tmp_txt = val_sim.strip()
        if re.match(r'[10]$', tmp_txt):
            v_s = bool(int(val_sim))
        else:
            raise
        v_t = bool(int(val_tdc))
        
    elif val_type=='SDTIME': 
        v_t = int(val_tdc.strip('ms'))
        units = 1000
        un_txt = val_sim.lower()
        un_txt = un_txt.split("[")[1]
        un_txt = un_txt.split("]")[0]
        if un_txt=="s":
            units = 1000
        elif un_txt=="min":
            units = 60000
        elif un_txt=="ms":
            units = 1
        tmp_txt = re.sub('[^0-9.E-]','', val_sim)
        v_s = int(math.floor(round_half_down(float(tmp_txt)*units,decimals=3)))
    elif val_type=='INT' or val_type=='DINT':
        v_t = int(val_tdc)
        fl_check = re.compile(r'\d+[.]\d+')
        if fl_check.search(val_sim):
            if '%' not in val_sim: 
                #print("Signal in Simadyn has REAL instead INT")
                return -2
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            #v_s = toDeci(tmp_txt.strip(),16)
            v_s = twos_complement(tmp_txt.strip(),16)
        elif '%' in val_sim:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = int(round(float(tmp_txt)*16384*0.01))
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = twos_complement(tmp_txt.strip(),16,base = 2)
            #print(v_s)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = int(val_sim)

    elif val_type=='WORD' or val_type=='DWORD':
        #print("TDC: ",val_tdc)
        #print("SIM: ",val_sim)
        tmp_txt = val_tdc.split('16#')[1]
        v_t = toDeci(tmp_txt.strip(),16)
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = toDeci(tmp_txt.strip(),2)
        else:
            tmp_txt = re.sub('[^0-9.E-]','', val_sim)
            v_s = int(tmp_txt)
    elif val_type=='BYTE':
        tmp_txt = val_tdc.split('16#')[1]
        v_t = toDeci(tmp_txt.strip(),16)
        if "H" in val_sim:
            tmp_txt = val_sim.split('0H')[1]
            v_s = toDeci(tmp_txt.strip(),16) 
        elif "B" in val_sim:
            tmp_txt = val_sim.split('0B')[1]
            tmp_txt =  tmp_txt.replace(" ", "")
            v_s = toDeci(tmp_txt.strip(),2)
    elif val_type=='STRING':
        v_t = val_tdc.strip("'")
        v_s = val_sim.strip("'")
    #print("TDC: ",v_t)
    #print("SIM: ",v_s)
    return v_t==v_s




In [88]:
with open(FILE_NAME_DATA1.split(".")[0]+".txt", 'w') as f:
    print(COMPARATOR_README,file = f)
    data_a1_1 = data_a1
    for i, j in data_a1_1.iterrows():
        if(not j['out']) and not pd.isnull(j['value']):
            #print(j['rack'],j['chart'],j['block'],j['signal'],file = f)
            if not SKIP_TYPE_STR or j['type']!="STRING":
                value,subtype = get_some_value(j['rack'],j['chart'],j['block'],j['signal'],j['type'])
                if (value == -1):
                    if not SKIP_BLOCK_NEXST:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!\r\n",file = f)
                elif value == -2:
                    if not SKIP_SIGNAL_NFND:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!\r\n",file = f)
                elif value == -3:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL can not be recognized!!!!!!!!!!\r\n",file = f)
                elif value == -4:
                    if not SKIP_CHART_NFND:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!FILE not exist!!!!!!!!!!\r\n",file = f)
                else:
                    #try:
                    compare_result = compare_values(j['value'],value,j['type'],subtype)
                    if compare_result==-1:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL missing connection    XXXXXXXXXXXXXXXXXXXXXXXXXXX\r\n",file = f)
                    elif compare_result==-2:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX\r\n",file = f)
                    else:
                        if not compare_result or not SKIP_EQUAL:
                            print(j['rack'],j['chart'],j['block'],j['signal'], "Comparison:",j['value'], ">>>",value,">>>",compare_result,"\r\n",file = f)  

                    #except ValueError:
                    #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably float in INT field XXXXXXXXXXXXXXXXXXXXXXXXXXX",file = f)
                    #except UnboundLocalError:
                    #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably lost chart connection XXXXXXXXXXXXXXXXXXXXXXXX",file = f)

In [32]:
#data_a1[(data_a1['chart']=="RACK_1")&(data_a1['block']=="VM1")]
data_a1.head(-5)

,rack_cpu,chart,block,signal,value,type,out,rack,cpu
0,TX04\D01P01\,ENPAR1,FLASH2,Q,0,BOOL,True,TX04,D01P01
1,TX04\D01P01\,ENPAR1,VPAR01,I1,1,BOOL,False,TX04,D01P01
2,TX04\D01P01\,ENPAR1,VPAR01,I2,0,BOOL,False,TX04,D01P01
3,TX04\D01P01\,ENPAR1,VPAR01,I3,0,BOOL,False,TX04,D01P01
4,TX04\D01P01\,ENPAR1,VPAR01,I4,0,BOOL,False,TX04,D01P01
5,TX04\D01P01\,ENPAR1,VPAR01,I5,0,BOOL,False,TX04,D01P01
6,TX04\D01P01\,ENPAR1,VPAR01,I6,0,BOOL,False,TX04,D01P01
7,TX04\D02P02\,VISU_2,VIB110,T,2000ms,SDTIME,False,TX04,D02P02
8,TX04\D02P02\,VISU_2,VIB110,I01,0,BOOL,False,TX04,D02P02
9,TX04\D02P02\,VISU_2,VIB110,I02,0,BOOL,False,TX04,D02P02


In [79]:
print(COMPARATOR_README)
data_a1_1 = data_a1
for i, j in data_a1_1.iterrows():
    if(not j['out']) and not pd.isnull(j['value']):
        print(j['rack'],j['chart'],j['block'],j['signal'])
        if not SKIP_TYPE_STR or j['type']!="STRING":
            value,subtype = get_some_value(j['rack'],j['chart'],j['block'],j['signal'],j['type'])
            #print(subtype)
            if (value == -1):
                if not SKIP_BLOCK_NEXST:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!ERROR Block is not found in SimadynD software!!!!!!!!!!")
            elif value == -2:
                if not SKIP_SIGNAL_NFND:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL is not found in SimadynD software!!!!!!!!!!")
            elif value == -3:
                print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!SIGNAL can not be recognized!!!!!!!!!!")
            elif value == -4:
                if not SKIP_CHART_NFND:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "!!!!!!!!!!FILE not exist!!!!!!!!!!")
            else:
                #try:
                compare_result = compare_values(j['value'],value,j['type'],subtype)
                if compare_result==-1:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL missing connection    XXXXXXXXXXXXXXXXXXXXXXXXXXX")
                elif compare_result==-2:
                    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Expect INT but have REAL XXXXXXXXXXXXXXXXXXXXXXXXXXX")
                else:
                    if not compare_result or not SKIP_EQUAL:
                        print(j['rack'],j['chart'],j['block'],j['signal'], "Comparison:",j['value'], ">>>",value,">>>",compare_result,"")  

                #except ValueError:
                #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably float in INT field XXXXXXXXXXXXXXXXXXXXXXXXXXX")
                #except UnboundLocalError:
                #    print(j['rack'],j['chart'],j['block'],j['signal'], "XXXXXXXXXXXXXXXX ERROR CRITICAL Probably lost chart connection XXXXXXXXXXXXXXXXXXXXXXXX")       

Comparator TDC <=> SimadynD software version: v.06 
Author: Anton Tushev 
Comparation result made from TX04_test.csv file 
Date and time: 2019-05-21 01:38:01.229993 
Use filters: 
XXXXXXXX - critical points 
!!!!!!!! - not critical errors 
False - difference in programs
True - no difference in programs 
Curren filters:
SKIP_EQUAL:		True
SKIP_BLOCK_NEXST:	True
SKIP_SIGNAL_NFND:	True
SKIP_TYPE_STR:		True
SKIP_CHART_NFND:	True

TX04 VISU_2 VIB110 I13
TX04 VISU_2 VIB110 I13 Comparison: 1 >>> 0 >>> False 
TX04 EXSCA2 DFGDF I2
